In [1]:
# Imports
from stable_baselines3 import PPO, DQN
import stable_baselines3.ppo.policies as ppo
import stable_baselines3.dqn.policies as dqn
from stable_baselines3.common.env_util import make_vec_env
import gym
import numpy as np
import random 
from gym.envs.box2d.lunar_lander import heuristic as lunar_heuristic
from stable_baselines3.common.monitor import Monitor
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv
from imitation.algorithms import bc
from stable_baselines3.common.evaluation import evaluate_policy
from parameter_initialization_deep_rl.common.helpers import create_folder
from stable_baselines3.common.callbacks import EvalCallback
from torch.nn.modules.activation import Sigmoid, Tanh, ReLU, LeakyReLU
from torch import nn
import torch as th

In [2]:
def heuristic_naive(obs: np.ndarray):
    obs = obs[0]
    action = 0
    if obs[1] < 0.3:
        if obs[3] < -0.05:
            action = 2
        else:
            action = 0
    else:
        a = random.random()
        if a < 0.4 and obs[3] < 0:
            action = 2
        else:
            if obs[4] < -0.1:
                action = 1
            elif obs[4] > 0.1:
                action = 3
    action = np.full((1),action)
    return action

def heuristic_expert(o: np.ndarray):
    # o is single observation and of shape (1,8)
    # action a has to be of shape (1)
    a = np.full((1),lunar_heuristic(gym.make("LunarLander-v2"), o[0]))
    return a

In [3]:
seeds = [95,  39,   1,  38,  83,  38,  36,  50,  14, 100,  67,  34,  68, 47,  70,  57,  64,  98,  93,  25]

In [4]:
log_dir = "./logs/dqn/bc_naive"

In [5]:
# Hyperparameter
hyperparameter = dict(
    n_steps = 16,
    gae_lambda = 0.98,
    gamma = 0.99,
    n_epochs = 4,
    ent_coef = 0.0,
)

In [6]:
hyperparameter = dict(
    learning_rate = 6.3e-4,
    batch_size = 128,
    buffer_size = 50000,
    learning_starts = 0,
    gamma = 0.99,
    target_update_interval = 250,
    train_freq = 4,
    gradient_steps = -1,
    exploration_fraction = 0.12,
    exploration_final_eps = 0.1,
)

In [ ]:
reward_sum = 0

for seed in seeds:
    # create environments and set seed
    env_train = gym.make("LunarLander-v2")
    env_train.seed(seed)
    env_test = Monitor(gym.make("LunarLander-v2"))
    env_test.seed(seed)
    
    # Collect rollouts using the expert
    rollouts = rollout.rollout(
        heuristic_naive,
        DummyVecEnv([lambda: RolloutInfoWrapper(env_train)]),
        rollout.make_sample_until(min_timesteps=None, min_episodes=200),
    )
    # Flatten the trajectories to obtain individual transitions
    transitions = rollout.flatten_trajectories(rollouts)
    # Set up student
    student = DQN(
        policy=dqn.MlpPolicy,
        env=env_train,
        policy_kwargs = dict(
            net_arch = [32,32]
        ),
        seed=seed,
        verbose=False,
        **hyperparameter,
    )
    # Get the layers of the students q_net
    student_policy = student.policy
    student_layers = [module for module in student.q_net.modules(
        ) if isinstance(module, nn.Linear)]
    # Set up behavior cloning agent
    bc_trainer = bc.BC(
        observation_space=env_train.observation_space,
        action_space=env_train.action_space,
        demonstrations=transitions,
    )
    # Pretrain the policy with behavior cloning
    bc_trainer.train(n_epochs=2)
    # Get network parameters of bc expert after training
    policy = bc_trainer.policy
    expert_shared_layers = [module for module in policy.mlp_extractor.modules(
        ) if isinstance(module, nn.Linear)]
    expert_output_layers = [module for module in policy.action_net.modules(
        ) if isinstance(module, nn.Linear)]
    expert_layers = expert_shared_layers + expert_output_layers
    # Copy expert parameters to student q_net of student
    for (student_layer, expert_layer) in zip(student_layers, expert_layers):
        with th.no_grad():
            student_layer.weight.copy_(expert_layer.weight)
            student_layer.bias.copy_(expert_layer.bias)
    student_policy.q_net_target.load_state_dict(student_policy.q_net.state_dict())
    # Test performance before training (after pretraining)
    reward_before_training, _ = evaluate_policy(student.policy, env_test, 20)
    reward_sum += reward_before_training
    print(f"0 Timesteps: {reward_before_training}")
    # Create log folder
    trial_log_dir = create_folder(f"{log_dir}/{seed}")
    # Create eval callback
    eval_callback = EvalCallback(
        env_test,
        log_path=trial_log_dir,
        n_eval_episodes=20,
        eval_freq=3750,
        deterministic=True,
        render=False
    )
    # Train the pretrained policy using the regular learning algorithm
    student.learn(
        total_timesteps=1.5e5,
        callback=eval_callback
    )
np.savez(f"{log_dir}/zero_scores", scores=[reward_sum])

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00139 |
|    entropy        | 1.39     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 40.5     |
|    loss           | 1.39     |
|    neglogp        | 1.39     |
|    prob_true_act  | 0.25     |
|    samples_so_far | 32       |
--------------------------------


494batch [00:01, 396.81batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000849 |
|    entropy        | 0.849     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 55.4      |
|    loss           | 1.16      |
|    neglogp        | 1.16      |
|    prob_true_act  | 0.454     |
|    samples_so_far | 16032     |
---------------------------------


934batch [00:02, 391.93batch/s]
974batch [00:02, 387.61batch/s]

...

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 1000      |
|    ent_loss       | -0.000901 |
|    entropy        | 0.901     |
|    epoch          | 1         |
|    l2_loss        | 0         |
|    l2_norm        | 67.3      |
|    loss           | 0.797     |
|    neglogp        | 0.797     |
|    prob_true_act  | 0.525     |
|    samples_so_far | 32032     |
---------------------------------


1468batch [00:04, 368.22batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 1500      |
|    ent_loss       | -0.000806 |
|    entropy        | 0.806     |
|    epoch          | 1         |
|    l2_loss        | 0         |
|    l2_norm        | 82.9      |
|    loss           | 0.736     |
|    neglogp        | 0.736     |
|    prob_true_act  | 0.547     |
|    samples_so_far | 48032     |
---------------------------------


1766batch [00:04, 368.71batch/s]
1794batch [00:04, 362.83batch/s]


0 Timesteps: -280.685424
Eval num_timesteps=3750, episode_reward=-201.17 +/- 32.41
Episode length: 1000.00 +/- 0.00
New best mean reward!
Eval num_timesteps=7500, episode_reward=-123.88 +/- 78.31
Episode length: 386.30 +/- 150.24
New best mean reward!
Eval num_timesteps=11250, episode_reward=-193.07 +/- 72.92
Episode length: 324.95 +/- 150.75
Eval num_timesteps=15000, episode_reward=-187.12 +/- 35.38
Episode length: 339.55 +/- 112.22
Eval num_timesteps=18750, episode_reward=-156.22 +/- 63.14
Episode length: 566.20 +/- 265.76
Eval num_timesteps=22500, episode_reward=-53.20 +/- 30.54
Episode length: 990.55 +/- 41.19
New best mean reward!
Eval num_timesteps=26250, episode_reward=-46.58 +/- 19.76
Episode length: 1000.00 +/- 0.00
New best mean reward!
Eval num_timesteps=30000, episode_reward=-52.96 +/- 17.08
Episode length: 1000.00 +/- 0.00
Eval num_timesteps=33750, episode_reward=-137.89 +/- 50.14
Episode length: 549.65 +/- 251.62
Eval num_timesteps=37500, episode_reward=-100.38 +/- 47.76


In [ ]:
from parameter_initialization_deep_rl.common.evaluate import (
    create_numpy_arr_from_logs,
    create_sample,
    mean_confidence_intervals
)
from parameter_initialization_deep_rl.common.helpers import plot_performance

log_dirs = [f"{log_dir}/{seed}/evaluations.npz" for seed in seeds]
avg_returns, n = create_numpy_arr_from_logs(log_dirs)

sample = create_sample(avg_returns)

# Save the performance score for the individual seeds
np.savez(f"{log_dir}/avg_perf_across_seeds",
            sample=sample)

performance_score = np.average(sample)

# Save the performance score, i.e., the averaged return across all seeds and evaluation periods, i.e., just one single number
np.savez(f"{log_dir}/perf_score",
            perf_score=performance_score)

AVG, H = mean_confidence_intervals(avg_returns, n)

reward_before_training = reward_sum / len(seeds)
AVG = np.append(reward_before_training, AVG)

# Save the average return and confidence intervals for all the individual evaluation trials averaged across all seeds
np.savez(f"{log_dir}/avg_perf_eval_trials", avg=AVG,
            h=H)

plot_performance(
    title="Performance",
    graph_label="Random",
    x = np.arange(0,62*16384,16364),
    y=AVG,
    x_label="timesteps",
    y_label="Return"
)

In [ ]:
print(AVG)